In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from feature_engine.creation import CyclicalFeatures
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, SplineTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import Ridge, PoissonRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.model_selection import TimeSeriesSplit, cross_validate, cross_val_score

from sklearn.metrics import mean_poisson_deviance, mean_squared_error

from sklearn.utils import gen_even_slices

## Load the data

link : https://www.kaggle.com/datasets/hmavrodiev/london-bike-sharing-dataset

or you can use :

<pre>
from sklearn.datasets import fetch_openml

bike_sharing = fetch_openml(
    "Bike_Sharing_Demand", version=2, as_frame=True, parser="pandas"
)
df = bike_sharing.frame
</pre>


* "`cnt`" - the count of a new bike shares
* "`t1`" - real temperature in C
* "`t2`" - temperature in C "feels like"
* "`hum`" - humidity in percentage
* "`windspeed`" - wind speed in km/h
* "`weathercode`" - category of the weather
* "`isholiday`" - boolean field - 1 holiday / 0 non holiday
* "`isweekend`" - boolean field - 1 if the day is weekend
* "`season`" - category field meteorological seasons: 0-spring ; 1-summer; 2-fall; 3-winter.

#### `weathe_code`" category description:
* 1 = Clear ; mostly clear but have some values with haze/fog/patches of fog/ fog in vicinity 
* 2 = scattered clouds / few clouds 
* 3 = Broken clouds 
* 4 = Cloudy 
* 7 = Rain/ light Rain shower/ Light rain 
* 10 = rain with thunderstorm 
* 26 = snowfall 
* 94 = Freezing Fog


#### **objective** : predict the future bike shares.

In [2]:
df = pd.read_csv('./data/london_bikes.csv')
df.head()

,cnt,t1,hum,wind_speed,weather_code,is_holiday,is_weekend,season,hour
0,182,3.0,93.0,6.0,3,0,1,3,0
1,138,3.0,93.0,5.0,1,0,1,3,1
2,134,2.5,96.5,0.0,1,0,1,3,2
3,72,2.0,100.0,0.0,1,0,1,3,3
4,47,2.0,93.0,6.5,1,0,1,3,4


## Feature and Target

In [3]:
y = df.pop('cnt')
X = df

## Cyclic features

There are some features that are cyclic by nature. 
* For example the hours of a day or the months in a year. 

In these cases, the higher values of the variable are closer to the lower values. 
* For example, December `(12)` is closer to January `(1)` than to June `(6)`. 

By applying a cyclical transformations, that is, with the sine and cosine transformations of the original variables, we can capture the cyclic nature and obtain a better representation of the proximity between values.


In [19]:
plt.figure(figsize=(15,3))
h = df['hour'].value_counts()
plt.bar(h.index, h.values, color='salmon')
plt.xticks(h.index, labels=h.index);
plt.title('Hour Feature value counts');

<img src='./plots/hour-val-counts.png'>

In [30]:
# using feature engine
cyclic = CyclicalFeatures(drop_original=True)
c = cyclic.fit_transform(df[['hour']])
c.head()

,hour_sin,hour_cos
0,0.000000,1.000000
1,0.269797,0.962917
2,0.519584,0.854419
3,0.730836,0.682553
4,0.887885,0.460065


In [39]:
plt.scatter(df['hour'], c['hour_sin'], label='sine')
plt.scatter(df['hour'], c['hour_cos'], label='cosine')
plt.title('Cyclical Features')
plt.legend();

<img src='./plots/cyclic-feat.png'>


## Periodic spline : Modeling seasonal effects

* Seasonal effects can be modelled using periodic splines, which have equal function value and equal derivatives at the first and last knot.

* The splines period is the distance between the first and last knot (which we specify manually, if known)

* Periodic splines provide a better fit both within and outside of the range of training data given the additional information of periodicity. 

* Periodic splines can also be useful for naturally periodic features (such as day of the year), as the smoothness at the boundary knots prevents a jump in the transformed values (e.g. from Dec 31st to Jan 1st). 

* For naturally periodic features or more generally features where the period is known, it is advised to explicitly pass this information to the SplineTransformer by setting the knots manually.


In [90]:
# using spline transformer from sklearn

spline = SplineTransformer(degree=3, n_knots=13, extrapolation='periodic')
temp_df = pd.DataFrame(data=spline.fit_transform(df[['hour']]), columns=[f'spline-{i}' for i in range(12)])
temp_df.head()


,spline-0,spline-1,spline-2,spline-3,spline-4,spline-5,spline-6,spline-7,spline-8,spline-9,spline-10,spline-11
0,0.166667,0.666667,0.166667,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.018232,0.465467,0.492630,0.023671,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.145859,0.664817,0.189310,0.000014,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.013698,0.437481,0.518726,0.030095,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.126860,0.659434,0.213597,0.00011,0.0,0.0,0.0,0.0,0.0,0.0


In [92]:
temp_df['hour'] = df['hour']
temp_df.plot(x='hour', linestyle='', marker='.')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0);

<img src='./plots/spline-feat-plot-1.png'>

In [77]:
# if we know the periodicity we should provide that info to spline transformer
n_knots = 13
period = 24
knots = (np.linspace(0, period, n_knots))[:,np.newaxis]
spline = SplineTransformer(degree=3, n_knots=13, knots=knots, extrapolation='periodic')
temp_df = pd.DataFrame(data=spline.fit_transform(df[['hour']]), columns=[f'spline-{i}' for i in range(12)])
temp_df.head()

,spline-0,spline-1,spline-2,spline-3,spline-4,spline-5,spline-6,spline-7,spline-8,spline-9,spline-10,spline-11
0,0.166667,0.666667,0.166667,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.020833,0.479167,0.479167,0.020833,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.166667,0.666667,0.166667,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.020833,0.479167,0.479167,0.020833,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.166667,0.666667,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [93]:
for col in temp_df.columns:
    plt.plot(df['hour'][:24], temp_df[col][:24])


<img src='./plots/spline-feat-plot-2.png'>

## Categorical Features

In [4]:
cat_features = ['weather_code',	'is_holiday','is_weekend', 'season']

In [182]:
df[cat_features].head()

,weather_code,is_holiday,is_weekend,season
0,3,0,1,3
1,1,0,1,3
2,1,0,1,3
3,1,0,1,3
4,1,0,1,3


### Let's study at the categories and decide how to encode it

In [183]:
def feature_stats(name):
    print(f'FEATURE : {name}')
    counts = df[name].value_counts().values
    unique_vals = df[name].value_counts().index
    print(f'Number of unique values : {len(unique_vals)}')
    print(f'Unique values : {unique_vals}')
    print(f'Counts : {counts}')
    print('-'*100)

feature_stats('weather_code')
feature_stats('is_holiday')
feature_stats('is_weekend')
feature_stats('season')

FEATURE : weather_code
Number of unique values : 7
Unique values : Int64Index([1, 2, 3, 7, 4, 26, 10], dtype='int64')
Counts : [6150 4034 3551 2141 1464   60   14]
----------------------------------------------------------------------------------------------------
FEATURE : is_holiday
Number of unique values : 2
Unique values : Int64Index([0, 1], dtype='int64')
Counts : [17030   384]
----------------------------------------------------------------------------------------------------
FEATURE : is_weekend
Number of unique values : 2
Unique values : Int64Index([0, 1], dtype='int64')
Counts : [12444  4970]
----------------------------------------------------------------------------------------------------
FEATURE : season
Number of unique values : 4
Unique values : Int64Index([0, 1, 3, 2], dtype='int64')
Counts : [4394 4387 4330 4303]
----------------------------------------------------------------------------------------------------


### Let's onehot encode the categorical features

In [184]:
onehot = OneHotEncoder(handle_unknown='ignore')
encoding = onehot.fit_transform(df[cat_features])
encoding.toarray()

array([[0., 0., 1., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.]])

## The numerical features
* `t1`	
* `hum`	
* `wind_speed`

Let's scale them using `MinMaxScaler`

In [185]:
num_features = ['t1', 'hum', 'wind_speed']

In [186]:
df[num_features].head()

,t1,hum,wind_speed
0,3.0,93.0,6.0
1,3.0,93.0,5.0
2,2.5,96.5,0.0
3,2.0,100.0,0.0
4,2.0,93.0,6.5


In [187]:
scaler = MinMaxScaler()

scaler.fit_transform(df[num_features])

array([[0.12676056, 0.91194969, 0.10619469],
       [0.12676056, 0.91194969, 0.08849558],
       [0.11267606, 0.95597484, 0.        ],
       ...,
       [0.1971831 , 0.72955975, 0.42477876],
       [0.1971831 , 0.69811321, 0.40707965],
       [0.18309859, 0.69811321, 0.38938053]])

## Building model pipeline

In [5]:
preprocess = ColumnTransformer(transformers=[
    ('cyclic-transform',SplineTransformer(n_knots=13, extrapolation='periodic') , ['hour']),  
    ('one-hot-encode', OneHotEncoder(handle_unknown='ignore'), ['weather_code',	'is_holiday','is_weekend', 'season']),
], remainder=MinMaxScaler())

In [6]:
def build_pipeline(model, preprocess):
    pipe = Pipeline(steps=[
    ('preprocessing', preprocess),
    ('modeling', model)
    ])
    return pipe

In [190]:
ridge_pipe = build_pipeline(model=Ridge(), preprocess=preprocess)

ridge_pipe.fit(X, y)

print('Ridge pipeline score on training data:',ridge_pipe.score(X, y))

print('Mean Cross validation score ',cross_val_score(ridge_pipe, X, y).mean())

Ridge pipeline score on training data: 0.7007001700388873
Mean Cross validation score  0.6691829612247873


In [191]:
poisson_pipe = build_pipeline(model=PoissonRegressor(max_iter=300), preprocess=preprocess)

poisson_pipe.fit(X, y)

print('Poisson pipeline score on training data:',poisson_pipe.score(X, y))
print('Mean Cross validation score ',cross_val_score(poisson_pipe, X, y).mean())

Poisson pipeline score on training data: 0.8046505299510963
Mean Cross validation score  0.7907142156991612


## Cross validation

In [7]:
def custom_metric(est, X, y):
    y_pred = est.predict(X)
    mask = y_pred>0
    return {
        'mean_poisson_deviance' : mean_poisson_deviance(y[mask], y_pred[mask]),
        'root_mean_squared_error' : mean_squared_error(y, y_pred, squared=False)
    }
    

### Time Series cross-validator

In [8]:
tscv = TimeSeriesSplit(n_splits=50, max_train_size=10000, test_size=336)

ridge_pipe = build_pipeline(model=Ridge(), preprocess=preprocess)

ridge_cv_result = cross_validate(ridge_pipe, X, y, cv=tscv, scoring=custom_metric, return_estimator=True)

print('Mean poisson deviance : ',ridge_cv_result['test_mean_poisson_deviance'].mean())
print('Mean squared error : ',ridge_cv_result['test_root_mean_squared_error'].mean())

Mean poisson deviance :  274.0353425299007
Mean squared error :  594.7947793289886


In [9]:
def cross_validate_util(est):
    tscv = TimeSeriesSplit(n_splits=50, max_train_size=10000, test_size=336)

    result = cross_validate(est, X, y, cv=tscv, scoring=custom_metric, return_estimator=True)

    print('Mean poisson deviance : ',result['test_mean_poisson_deviance'].mean())
    print('Mean squared error : ',result['test_root_mean_squared_error'].mean())
    return result 


### Create a test-set using the test-id's generated by Time Series cross-validator

In [10]:
X_test, y_test = [], []
for train_id, test_id in tscv.split(X, y):
    test_x = X.iloc[test_id]
    test_y = y.iloc[test_id]

    X_test.append(test_x)
    y_test.append(test_y)


# lets concat the y_test into one series
y_test = pd.concat(y_test, axis=0)

how would we generate a prediction for the test splits ?
* Use the estimator returned from `cross_validate( ...,  return_estimator=True)`
* Iterate  `zip(X_test, result_obj['estimator'])`
* make the prediction for every test-split
* finally concatenate


In [11]:
# here we generate predictions and concat the y_preds into one series
def get_test_preds(result_obj):
    predictions = []
    for x, est in zip(X_test, result_obj['estimator']):
        y_pred = est.predict(x)
        predictions.append(y_pred)
    
    return np.concatenate(predictions, axis=0)

In [12]:
# utility that build, cross-validate & test 
def evaluate(model, preprocess):
    pipe = build_pipeline(model, preprocess)
    result = cross_validate_util(pipe)
    y_preds = get_test_preds(result)
    return {**result, 'test_prediction':y_preds}

## Ridge Regression

In [14]:
ridge_result = evaluate(Ridge(), preprocess)

Mean poisson deviance :  274.0353425299007
Mean squared error :  594.7947793289886


In [209]:
ridge_result.keys()

dict_keys(['fit_time', 'score_time', 'estimator', 'test_mean_poisson_deviance', 'test_root_mean_squared_error', 'test_prediction'])

In [263]:
plt.figure(figsize=(15,4))
plt.subplot(121)
sns.swarmplot(x=ridge_result['test_mean_poisson_deviance'] , color=".25" )
sns.boxplot(ridge_result['test_mean_poisson_deviance'], orient='h', color='lightblue');
plt.title('mean_poisson_deviance')
plt.subplot(122)
sns.swarmplot(x=ridge_result['test_root_mean_squared_error'] , color=".25" )
sns.boxplot(ridge_result['test_root_mean_squared_error'], orient='h', color='lightblue');
plt.title('root_mean_squared_error')

plt.suptitle('Ridge Regression\n',size=24);


<img src='./plots/ridge-model-performance-plot.png'>

## Poisson Regression

In [15]:
poisson_result = evaluate(PoissonRegressor(max_iter=300), preprocess)

Mean poisson deviance :  193.43627484439133
Mean squared error :  527.8672054124788


In [265]:
plt.figure(figsize=(15,4))
plt.subplot(121)
sns.swarmplot(x=poisson_result['test_mean_poisson_deviance'] , color=".25" )
sns.boxplot(poisson_result['test_mean_poisson_deviance'], orient='h', color='lightblue');
plt.title('mean_poisson_deviance')
plt.subplot(122)
sns.swarmplot(x=poisson_result['test_root_mean_squared_error'] , color=".25" )
sns.boxplot(poisson_result['test_root_mean_squared_error'], orient='h', color='lightblue');
plt.title('root_mean_squared_error')

plt.suptitle('Poisson Regression\n', size=24);

<img src='./plots/poisson-model-performance-plot.png'>

## Comparsion Ridge vs Poisson

In [266]:
temp_a = pd.DataFrame(data={
    'model' : 'Ridge',
    'mean_poisson_deviance' : ridge_result['test_mean_poisson_deviance'],
    'root_mean_squared_error' : ridge_result['test_root_mean_squared_error']
})

temp_b = pd.DataFrame(data={
    'model' : 'Poisson',
    'mean_poisson_deviance' : poisson_result['test_mean_poisson_deviance'],
    'root_mean_squared_error' : poisson_result['test_root_mean_squared_error']
})

temp_df = pd.concat([temp_a, temp_b])

plt.figure(figsize=(15,4))
plt.subplot(121)
sns.boxplot(data=temp_df, x='mean_poisson_deviance', y='model');
sns.swarmplot(data=temp_df, x='mean_poisson_deviance', y='model', color='black');

plt.subplot(122)
sns.boxplot(data=temp_df, x='root_mean_squared_error', y='model');
sns.swarmplot(data=temp_df, x='root_mean_squared_error', y='model', color='black');

plt.suptitle('Comparing linear model performance on count data\n We can see that the Possion regression is better for count data\n');

<img src='./plots/comparing-linear-models-on-count-data.png'>

In [16]:
# utility for building comparison plots --

def compare_model_performance_on_count_data(results, n_rows=1, n_cols=2, figsize=(15,4)):
    # create a dataframe
    temp_df = pd.concat([

        pd.DataFrame(data={
        'model' : k,
        'mean_poisson_deviance' : v['test_mean_poisson_deviance'],
        'root_mean_squared_error' : v['test_root_mean_squared_error']
        })

        for k,v in results.items()
    ])

    # plots
    fig, ax = plt.subplots(ncols=n_cols, nrows=n_rows, figsize=figsize)
    fields = ['mean_poisson_deviance', 'root_mean_squared_error']
    for i, frame in enumerate(ax.ravel()):
        sns.boxplot(data=temp_df, x=fields[i], y='model', ax=frame);
        sns.swarmplot(data=temp_df, x=fields[i], y='model', color='black', ax=frame);
        frame.set(title=fields[i])

    plt.suptitle('Model performance : '+' vs '.join([k for k in results.keys()]))
    plt.tight_layout()
    

In [315]:
# compare_model_performance_on_count_data({'ridge':ridge_result, 'poisson':poisson_result})

### Let's look at the distribution of predicted counts

In [27]:


fig, ax = plt.subplots(nrows=1, ncols=3, constrained_layout=True, figsize=(14,4), sharey=True)


sns.histplot(y_test, ax=ax[0]);
ax[0].set(title='True counts')


sns.histplot(ridge_result['test_prediction'], ax=ax[1]);
ax[1].set(title='Predicted counts (Ridge)')


sns.histplot(poisson_result['test_prediction'], ax=ax[2]);
ax[2].set(title='Predicted counts (Poisson)')



fig.suptitle('The poisson model made the best prediction\n', size=20)
plt.tight_layout()


<img src='./plots/Linear-models-prediction-distribution-plot.png'>

In [28]:
def plot_predictions(results, nrows=1, ncols=3, figsize=(15,4)):

    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize, constrained_layout=True, sharey=True)
    ax = ax.ravel() 

    sns.histplot(y_test, ax=ax[0]);
    ax[0].set(title='True counts')

    for i, (k, v) in enumerate(results.items()):
        sns.histplot(v['test_prediction'], ax=ax[i+1]);
        ax[i+1].set(title=f'Predicted counts ({k})')
            

    fig.suptitle('Distribution plot : y_true vs model_prediction\n', size=20)
    plt.tight_layout()

In [335]:
# plot_predictions(results={'ridge':ridge_result, 'poisson':poisson_result})

## Calibration for Regression

In [13]:
def calibration_plot(y_true, y_pred, title, ax, n_bins=10):
    # step 1 : get sort id using y_pred
    id_sort = np.argsort(y_pred)
    y_true = np.asarray(y_true)

    # step 2 : sort the y_pred and y_true using the sort-id from step-1 
    y_pred_sorted = y_pred[id_sort]
    y_true_sorted = y_true[id_sort]

    # step 3 : create placeholder arrays for storing summary
    y_pred_bin = np.empty(n_bins)
    y_true_bin = np.empty(n_bins)

    # step 4 : Generator to create n_packs evenly spaced slices going up to n
    for i, chunk in enumerate(gen_even_slices(n=len(y_pred), n_packs=n_bins)):
        # store the summary
        y_pred_bin[i] = np.mean(y_pred_sorted[chunk])
        y_true_bin[i] = np.mean(y_true_sorted[chunk])
    

    # step 5 : calculate the difference
    diff = y_pred_bin - y_true_bin

    # step 6 : plotting
    x = np.arange(n_bins)
    ax.plot(x, diff, marker='o')
    ax.axhline(y=0, c='k', linestyle='--')
    ax.set(title=title, xlabel='bins', ylabel='E(y_hat) - E(y_true)')
    

In [33]:

fig, ax = plt.subplots(1, 2, constrained_layout=True, figsize=(15, 6), dpi=300, sharey=True)

calibration_plot(y_test, ridge_result['test_prediction'], title='Ridge regression', ax=ax[0])

calibration_plot(y_test, poisson_result['test_prediction'], title='Poisson regression', ax=ax[1])

plt.suptitle('Calibration for Regression\n', size=20)
plt.tight_layout()


<img src='./plots/Calibration-plot-Possion-vs-Ridge.png'>

## Random Forest and Count data

* For count data use `(criterion='poisson')`

In [14]:
randomforest_result = evaluate(RandomForestRegressor(criterion='poisson'), preprocess)

Mean poisson deviance :  61.589812191419995
Mean squared error :  285.3950476646621


In [321]:
compare_model_performance_on_count_data(results={'Poisson': poisson_result, 'RandomForest':randomforest_result})

<img src='./plots/comparing-poisson-vs-randomforest-on-count-data.png'>


### Random forest has much more capacity so it can learn more about the data 

In [323]:
compare_model_performance_on_count_data(results={'Poisson': poisson_result, 'RandomForest':randomforest_result, 'Ridge':ridge_result})

<img src='./plots/comparing-poisson-vs-randomforest-vs-ridge-on-count-data.png'>

## Calibration for Regression : Poisson vs Random Forest

In [36]:
fig, ax = plt.subplots(1, 2, constrained_layout=True, figsize=(15, 6), dpi=300, sharey=True)

calibration_plot(y_test, randomforest_result['test_prediction'], title='Random Forest regression criterion = poisson', ax=ax[0])

calibration_plot(y_test, poisson_result['test_prediction'], title='Poisson regression', ax=ax[1])

plt.suptitle('Calibration for Regression : Poisson vs Random Forest\n', size=20)
plt.tight_layout()

<img src='./plots/Calibration-plot-for-Regression-Poisson-vs-Random Forest.png'>

In [343]:
plot_predictions(results={'Random Forest':randomforest_result, 'poisson':poisson_result})

<img src='./plots/distribution-plot-poisson-vs-randomforest.png'>

## Random Forest criterion 
`RandomForestRegressor(criterion='poisson')` **vs** `RandomForestRegressor(criterion='squared_error')`

In [37]:
randomforest_result_squared_error = evaluate(RandomForestRegressor(), preprocess)

Mean poisson deviance :  62.05426978471689
Mean squared error :  286.7635774474529


In [15]:
fig, ax = plt.subplots(1, 2, constrained_layout=True, figsize=(15, 6), dpi=300, sharey=True)

calibration_plot(y_test, randomforest_result['test_prediction'], title='Random Forest regression criterion = poisson', ax=ax[0])

calibration_plot(y_test, randomforest_result_squared_error['test_prediction'], title='Random Forest regression criterion = squared_error', ax=ax[1])

plt.suptitle('Calibration for Random Forest Regression  criterion: poisson vs squared-error\n Both underpredicts the larger counts', size=20)
plt.tight_layout()

<img src='./plots/Calibration-for-RandomForest-different-criterion.png'>

### We can see that both model underpredicts for larger counts

In [348]:
plot_predictions(results={'Random Forest Poisson':randomforest_result, 'Random Forest Squared error':randomforest_result_squared_error})

<img src='./plots/distribution-plot-randomforest-criterion.png'>

## HistGradientBoostingRegressor
* for count data use ` (loss='poisson')  `

In [16]:
from sklearn.ensemble import HistGradientBoostingRegressor

gbr_results = evaluate(HistGradientBoostingRegressor(loss='poisson', random_state=0, max_depth=4), preprocess)

Mean poisson deviance :  59.57934988161449
Mean squared error :  277.6426028293337


In [368]:
compare_model_performance_on_count_data(
    results={
    'Poisson': poisson_result,
    'RandomForest':randomforest_result, 
    'HistGradientBoostingRegressor':gbr_results})

<img src='./plots/comparing-poisson-vs-RandomForest-vs-GBR-on-count-data.png'>

In [366]:

plot_predictions(results={'Random Forest Poisson':randomforest_result,  'HistGradientBoostingRegressor':gbr_results})

<img src='./plots/distribution-plot-RandomForest-vs-GBR.png'>

In [18]:
fig, ax = plt.subplots(1, 2, constrained_layout=True, figsize=(15, 6), dpi=300, sharey=True)

calibration_plot(y_test, randomforest_result['test_prediction'], title='Random Forest regression criterion = poisson', ax=ax[0])

calibration_plot(y_test, gbr_results['test_prediction'], title='HistGradientBoostingRegressor', ax=ax[1])

plt.suptitle('Calibration plot: Random Forest Regression vs HistGradientBoostingRegressor\n Both underpredicts the larger counts', size=20)
plt.tight_layout()

<img src='./plots/Calibration-plot-for-RandomForest-vs-GBR-model.png'>

### Why are these models under-predicting for larger counts ?

In [394]:
# lets obtain a test split
splits = list((tscv.split(X, y)))

train_id, test_id = splits[-1]


train_x = X.iloc[train_id]
train_y = y.iloc[train_id]
test_x = X.iloc[test_id]
test_y = y.iloc[test_id]

In [397]:
# lets train the models
poisson = PoissonRegressor(max_iter=300)
rf = RandomForestRegressor(criterion='poisson')
gbr = HistGradientBoostingRegressor(loss='poisson')

poisson.fit(train_x, train_y)
rf.fit(train_x, train_y)
gbr.fit(train_x, train_y)

# make prediction
poisson_y_pred = poisson.predict(test_x)
rf_y_pred = rf.predict(test_x)
gbr_y_pred = gbr.predict(test_x)

In [399]:
temp_df = pd.DataFrame(data={
    'True_counts':test_y, 
    'Poisson':poisson_y_pred,
    'RandomForest':rf_y_pred,
    'GradientBoosting':gbr_y_pred
    })

In [418]:
def visualize_prediction(model, color='r', alpha=0.7):
    ax = temp_df.plot(y=['True_counts'], c='k', alpha=0.5)
    temp_df.plot(y=[model], ax=ax, c=color, alpha=alpha)
    ax.axes.xaxis.set_ticklabels([]);

## Under-prediction for larger counts 
* We can clearly see from to visuals that our models are under-predicting for larger count!
* May be we are missing some crucial features


In [423]:
visualize_prediction(model='Poisson')

<img src='./plots/model-under-pred-poisson.png'>

In [426]:
visualize_prediction(model='RandomForest', color='g', alpha=0.5)


<img src='./plots/model-under-pred-randomforest.png'>

In [427]:
visualize_prediction(model='GradientBoosting', color='b', alpha=0.5)

<img src='./plots/model-under-pred-GBR.png'>